# RAG pipeline

In [69]:
from langchain.llms import Ollama
from langchain.chains import RetrievalQA
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import FAISS
import pandas as pd
import psycopg2
from langchain.embeddings import HuggingFaceEmbeddings



In [70]:
# Step 1: Load Data from CSV
csv_path = "rag_data.csv"  # Replace with your CSV file path
loader = CSVLoader(csv_path)
data = loader.load()

In [71]:
print(data)

[Document(metadata={'source': 'rag_data.csv', 'row': 0}, page_content='id: 1\ntext: Artificial Intelligence (AI) refers to the simulation of human intelligence in machines that are programmed to think and act like humans.'), Document(metadata={'source': 'rag_data.csv', 'row': 1}, page_content='id: 2\ntext: Machine Learning is a subset of AI that involves training algorithms to learn from and make predictions or decisions based on data.'), Document(metadata={'source': 'rag_data.csv', 'row': 2}, page_content="id: 3\ntext: Deep Learning is a specialized form of Machine Learning that uses neural networks with many layers (hence 'deep') to analyze various data types."), Document(metadata={'source': 'rag_data.csv', 'row': 3}, page_content='id: 4\ntext: Natural Language Processing (NLP) is a field of AI that focuses on the interaction between computers and human language.'), Document(metadata={'source': 'rag_data.csv', 'row': 4}, page_content='id: 5\ntext: Reinforcement Learning is a type of 

In [72]:
# Step 2: Split Data into Documents
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_documents(data)

In [73]:
print(documents)

[Document(metadata={'source': 'rag_data.csv', 'row': 0}, page_content='id: 1\ntext: Artificial Intelligence (AI) refers to the simulation of human intelligence in machines that are programmed to think and act like humans.'), Document(metadata={'source': 'rag_data.csv', 'row': 1}, page_content='id: 2\ntext: Machine Learning is a subset of AI that involves training algorithms to learn from and make predictions or decisions based on data.'), Document(metadata={'source': 'rag_data.csv', 'row': 2}, page_content="id: 3\ntext: Deep Learning is a specialized form of Machine Learning that uses neural networks with many layers (hence 'deep') to analyze various data types."), Document(metadata={'source': 'rag_data.csv', 'row': 3}, page_content='id: 4\ntext: Natural Language Processing (NLP) is a field of AI that focuses on the interaction between computers and human language.'), Document(metadata={'source': 'rag_data.csv', 'row': 4}, page_content='id: 5\ntext: Reinforcement Learning is a type of 

In [74]:
# Step 3: Use Sentence-Transformers for Embeddings
# Load a pre-trained sentence transformer model
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Create embeddings for documents
# embeddings = embedding_model.encode([doc.page_content for doc in documents])

# Create FAISS vector store
vectorstore = FAISS.from_documents(documents, embeddings)


/Volumes/home/Learn/Master/3.LLMs/rag_pipeline/.venv/lib/python3.11/site-packages/torch/nn/modules/module.py:456: RuntimeWarning: coroutine 'FAISS.asimilarity_search_by_vector' was never awaited
  super().__setattr__('_forward_hooks_with_kwargs', OrderedDict())
/Volumes/home/Learn/Master/3.LLMs/rag_pipeline/.venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [75]:
# Step 4: Initialize LLaMA LLM via Ollama
llm = Ollama(model="llama3.1:latest")


In [76]:
# Step 5: Create the RAG Pipeline with LangChain's RetrievalQA Chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # You can also use "map_reduce" for larger datasets
    retriever=vectorstore.as_retriever()
)


In [77]:

# Step 6: Run a Q&A Test
query = "FPT Smart Cloud là gì"
result = qa_chain.run(query)
print(result)

According to id: 11, text: FPT smart cloud là 1 công ty con của FPT. 

Translation: FPT Smart Cloud is a subsidiary of FPT.


# Database integrate

In [78]:
import pandas as pd
df = pd.read_parquet('Văn Bản Pháp Luật_p100000-100019.parquet')
# Đọc file Parquet bằng pandas
parquet_file_path = 'Văn Bản Pháp Luật_p100000-100019.parquet'  # Đường dẫn đến file Parquet
df = pd.read_parquet(parquet_file_path)
df['created_date'] = pd.to_datetime(df['created_date'], format='%d/%m/%Y')
df['updated_date'] = pd.to_datetime(df['updated_date'], format='%d/%m/%Y')
df['content_raw'] =  df['content'].apply(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)
df.head()


,id,url,title,created_date,updated_date,content,content_raw
0,100000,https://thuvienphapluat.vn/van-ban/Quyen-dan-s...,Quyết định 720/QĐ-CTN năm 2020 về cho thôi quố...,2020-05-18,2020-06-03,"b'<div class=""content1"">\n <div>\n <div>\n ...","<div class=""content1"">\n <div>\n <div>\n <t..."
1,100001,https://thuvienphapluat.vn/van-ban/Quyen-dan-s...,Quyết định 719/QĐ-CTN năm 2020 về cho thôi quố...,2020-05-18,2020-06-03,"b'<div class=""content1"">\n <div>\n <div>\n ...","<div class=""content1"">\n <div>\n <div>\n <t..."
2,100002,https://thuvienphapluat.vn/van-ban/Xay-dung-Do...,Quyết định 648/QĐ-TTg năm 2020 về phê duyệt nh...,2020-05-18,2020-05-18,"b'<div class=""content1"">\n <div>\n <div>\n ...","<div class=""content1"">\n <div>\n <div>\n <t..."
3,100003,https://thuvienphapluat.vn/van-ban/Thuong-mai/...,Quyết định 1000/QĐ-UBND năm 2020 công bố danh ...,2020-05-18,2020-08-24,"b'<div class=""content1"">\n <div>\n <div>\n ...","<div class=""content1"">\n <div>\n <div>\n <t..."
4,100004,https://thuvienphapluat.vn/van-ban/Bo-may-hanh...,Quyết định 1517/QĐ-UBND năm 2020 về công bố Da...,2020-05-18,2020-09-30,"b'<div class=""content1"">\n <div>\n <div>\n ...","<div class=""content1"">\n <div>\n <div>\n <t..."


In [79]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   id            20 non-null     int64         
 1   url           20 non-null     object        
 2   title         20 non-null     object        
 3   created_date  20 non-null     datetime64[ns]
 4   updated_date  20 non-null     datetime64[ns]
 5   content       20 non-null     object        
 6   content_raw   20 non-null     object        
dtypes: datetime64[ns](2), int64(1), object(4)
memory usage: 1.2+ KB


In [80]:
# Kết nối tới PostgreSQL
def create_connection():
    connection = psycopg2.connect(
        host="localhost",
        port="5432",
        user="postgres",
        password="123456",
        database="db_llm"
    )
    return connection

In [81]:


connection = create_connection()

cursor = connection.cursor()

try: 
    # Chèn dữ liệu từ DataFrame vào PostgreSQL
    for index, row in df.iterrows():
        cursor.execute(
            """
            INSERT INTO page_content (type, id, url, title, created_date, updated_date, content, content_raw)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
            """,
            ("van_ban_phap_luat",row['id'], row['url'], row['title'], row['created_date'], row['updated_date'], row['content'], row['content_raw'])
        )

    # Xác nhận các thay đổi vào cơ sở dữ liệu
    connection.commit()

except Exception as e:
    print(e)

# Đóng kết nối
cursor.close()
connection.close()

print("Data inserted successfully!")


duplicate key value violates unique constraint "page_content_pkey"
DETAIL:  Key (type, id)=(van_ban_phap_luat, 100000) already exists.

Data inserted successfully!


In [82]:
connection = create_connection()
# Query to fetch data
query = "SELECT * FROM page_content;"

# Load data into a DataFrame
df = pd.read_sql_query(query, connection)

# Close the connection
connection.close()

df.head(2)

/var/folders/38/q1znsjsj5c17qlggflslw4000000gn/T/ipykernel_37903/1748144191.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,type,id,url,title,created_date,updated_date,content,content_raw
0,van_ban_phap_luat,100000,https://thuvienphapluat.vn/van-ban/Quyen-dan-s...,Quyết định 720/QĐ-CTN năm 2020 về cho thôi quố...,2020-05-18,2020-06-03,\x3c64697620636c6173733d22636f6e74656e7431223e...,"<div class=""content1"">\n <div>\n <div>\n <t..."
1,van_ban_phap_luat,100001,https://thuvienphapluat.vn/van-ban/Quyen-dan-s...,Quyết định 719/QĐ-CTN năm 2020 về cho thôi quố...,2020-05-18,2020-06-03,\x3c64697620636c6173733d22636f6e74656e7431223e...,"<div class=""content1"">\n <div>\n <div>\n <t..."


In [83]:

# Define the Document class
class Document:
    def __init__(self,page_content,metadata):
        self.page_content = page_content
        self.metadata = metadata
# Connect to PostgreSQL and fetch data
def fetch_data_from_postgres():
    connection = create_connection()
    query = "SELECT * FROM page_content;"
    df = pd.read_sql_query(query, connection)
    connection.close()
    return df

# Convert DataFrame to Document objects
def convert_df_to_documents(df):
    documents = [
        Document(
            page_content=f"ID: {row.id}, Title: {row.title}, Content: {row.content_raw}",  # or combine other fields if needed
            metadata={
                'id': row['id'],
                'url': row['url'],
                'title': row['title'],
                'created_date': row['created_date'],
                'updated_date': row['updated_date']
            }
        )
        for _, row in df.iterrows()
    ]
    return documents

# Fetch data
df = fetch_data_from_postgres()

# Convert to Document objects
documents = convert_df_to_documents(df)

# Example usage
for doc in documents:
    print(f"page_content: {doc.page_content}")  # 

page_content: ID: 100000, Title: Quyết định 720/QĐ-CTN năm 2020 về cho thôi quốc tịch Việt Nam đối với 95 công dân hiện đang cư trú tại Đức do Chủ tịch nước ban hành, Content: <div class="content1">
 <div>
  <div>
   <table border="0" cellpadding="0" cellspacing="0" style="border-collapse:collapse">
    <!--VABWAFAATABfADIAMAAyADMAMAA2ADAAOQA=-->
    <tr>
     <td style="width:167.4pt;padding:0in 5.4pt 0in 5.4pt" valign="top" width="223">
      <p align="center" style="margin-bottom:6.0pt;text-align:center">
       <b>
        CHỦ TỊCH NƯỚC
        <br/>
        --------
       </b>
      </p>
     </td>
     <td style="width:275.4pt;padding:0in 5.4pt 0in 5.4pt" valign="top" width="367">
      <p align="center" style="margin-bottom:6.0pt;text-align:center">
       <b>
        CỘNG HÒA XÃ
  HỘI CHỦ NGHĨA VIỆT NAM
        <br/>
        Độc lập - Tự do - Hạnh phúc
        <br/>
        ---------------
       </b>
      </p>
     </td>
    </tr>
    <tr>
     <td style="width:167.4pt;paddi

/var/folders/38/q1znsjsj5c17qlggflslw4000000gn/T/ipykernel_37903/1666297058.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


In [84]:
vectorstore = FAISS.from_documents(documents, embeddings)

In [85]:
# Step 5: Create the RAG Pipeline with LangChain's RetrievalQA Chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # You can also use "map_reduce" for larger datasets
    retriever=vectorstore.as_retriever()
)

In [86]:



# Step 6: Run a Q&A Test
query = "Quyết định 720/QĐ-CTN năm 2020 về cho thôi quốc tịch Việt Nam đối với 95 công dân hiện đang cư trú tại Đức do Chủ tịch nước ban hành"
result = qa_chain.run(query)
print(result)


Quyết định 720/QĐ-CTN năm 2020 về cho thôi quốc tịch Việt Nam đối với 95 công dân hiện đang cư trú tại Đức do Chủ tịch nước ban hành.


# Q&A 

In [87]:
from fuzzywuzzy import process


In [88]:
def get_faq_data():
    # Database connection parameters
    conn = create_connection()
    
    # Query to fetch data
    query = "SELECT question, answer FROM faq;"
    
    # Fetch data into a DataFrame
    df = pd.read_sql_query(query, conn)
    
    # Close the connection
    conn.close()
    
    return df

df = get_faq_data()
print(df.head())

                               question  \
0            What is the return policy?   
1           How do I reset my password?   
2             What are the store hours?   
3    Where can I find my order history?   
4  Do you offer international shipping?   

                                              answer  
0   You can return items within 30 days of purchase.  
1         Click 'Forgot Password' on the login page.  
2  We are open from 9 AM to 9 PM, Monday to Satur...  
3  Log in to your account and go to 'Order History'.  
4          Yes, we ship to many countries worldwide.  


/var/folders/38/q1znsjsj5c17qlggflslw4000000gn/T/ipykernel_37903/3740765136.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


In [106]:

# Convert DataFrame to Document objects
def convert_df_to_documents(df):
    documents = [
        Document(
            page_content=f"question: {row.question}",  # or combine other fields if needed
            metadata={
                'question': row['question'],
                'answer': row['answer'],
            }
        )
        for _, row in df.iterrows()
    ]
    return documents


In [107]:
question_documents = convert_df_to_documents(df)

In [108]:
from sentence_transformers import SentenceTransformer, util
import numpy as np
import torch

# Initialize HuggingFaceEmbeddings
model_name = "sentence-transformers/all-MiniLM-L6-v2"
embedding_model = HuggingFaceEmbeddings(model_name=model_name)

# Initialize FAISS vector store
vector_store = FAISS.from_documents(question_documents, embedding_model)


/Volumes/home/Learn/Master/3.LLMs/rag_pipeline/.venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [115]:
def search_similar_question(user_query):

    # Find the most similar question
    search_results = vector_store.similarity_search(user_query)
    
    
    return [{'question': item.metadata['question'],'answer': item.metadata['answer'] } for item in search_results]
    
    # else:
    #     return "Sorry, I couldn't find a similar question."


## Search 

In [116]:
# Example usage
user_query = "delivery"
search_similar_question(user_query)

[{'question': 'Do you offer international shipping?',
  'answer': 'Yes, we ship to many countries worldwide.'},
 {'question': 'What are the store hours?',
  'answer': 'We are open from 9 AM to 9 PM, Monday to Saturday.'},
 {'question': 'Where can I find my order history?',
  'answer': "Log in to your account and go to 'Order History'."},
 {'question': 'What is the return policy?',
  'answer': 'You can return items within 30 days of purchase.'}]